<a href="https://colab.research.google.com/github/stenforsm/predict-stock-news/blob/main/sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
df = pd.read_csv('b_news.csv') # choose which company file to work with here
x = df.title.values # process full headline
price = df["close price change"].values

In [3]:
# class_plus = 1; class_minus = 0
y = np.sign(price) # labels extracted by checking if +ve or -ve
y = np.where(y==1,1,0)
weight = np.abs(price) # sample weight is absolute price change value

In [4]:
# test train split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, w_train, w_test = train_test_split(x, y, weight, test_size=0.2)

In [5]:
# text vectorization

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
vectorize_layer.adapt(x_train)

def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)

x_train = vectorize_text(x_train)
x_test = vectorize_text(x_test)

In [7]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dense(1, activation='softmax')])

model.summary()

class_weight = {0: 1.,
                1: 50.} # this can be adjusted

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

model.fit(x_train, y_train, class_weight=class_weight, sample_weight=w_train, epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


191/191 [==============================] - 1s 5ms/step - loss: 9.7994 - binary_accuracy: 0.3888
Epoch 2/5
191/191 [==============================] - 1s 4ms/step - loss: 3.4787 - binary_accuracy: 0.3888
Epoch 3/5
191/191 [==============================] - 1s 4ms/step - loss: 2.4020 - binary_accuracy: 0.3888
Epoch 4/5
191/191 [==============================] - 1s 4ms/step - loss: 2.2826 - binary_accuracy: 0.3888
Epoch 5/5
191/191 [==============================] - 1s 4ms/step - loss: 2.2685 - binary_accuracy: 0.3888


In [9]:
model.evaluate(x_test, y_test)

48/48 [==============================] - 0s 1ms/step - loss: 2.2829 - binary_accuracy: 0.3916


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


[2.282865047454834, 0.3916175365447998]

In [10]:
predictions = model.predict(x_test)
y_predict = np.argmax(predictions, axis=1)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_predict))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, y_predict))

[[929   0]
 [598   0]]
0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
